![Image text](картинка.png)

In [ ]:
from sympy import randprime, mod_inverse

def generate_rsa_keys(bits=2048):
    # Генерация двух простых чисел p и q (должны быть ~1024 бита каждый)
    p = randprime(2**(bits//2 - 1), 2**(bits//2))
    q = randprime(2**(bits//2 - 1), 2**(bits//2))
    
    # Вычисление модуля n и функции Эйлера
    n = p * q
    phi = (p - 1) * (q - 1)
    
    # Выбор публичной экспоненты и вычисление приватной
    e = 65537
    d = mod_inverse(e, phi)
    
    return (e, n), (d, p, q)

public_key, private_key = generate_rsa_keys()
e, n = public_key
d, p, q = private_key
print(f"Public key: {public_key}")
print(f"Private key: {private_key}")

Public key: (65537, 17368357751069328306276600205786520939912782881737245874377219438347975687977783304230571192453231817920626971035237774673660035041216840037615813959618165617226881626371589842270280593162338482191760611710182252763885010950336527059296891455189409820486562471482097137630789587965016844253964032667680393274234600824166210800205745704024910790123943064677485090691917691993011117514276574181537105733518523616738065013507382049516270047013209862419036782571790870974798740314292302542617260052039524647032700810486495638654342226791417691113360691225740646557607149176597633434620755605330562282419037935670887272891)
Private key: (15561213214326087850953650775643879587863325835022164628403521231989813943980949388251074649106136470767099111726919017551733676206572934576326737020017081171129713525753193041464002561443248714425985315141042182550159453026257842071925704143702210443556006152257286414199684194060299663715218274717806084078623021531067674428269499139094009042979

Этот код гарантирует получение двух 1024-битных простых чисел, чье произведение даст 2048-битный модуль n.

In [39]:
from sympy import randprime, mod_inverse
import timeit

def generate_rsa_keys(bits=2048):
    """Генерация RSA ключей для демонстрации"""
    p = randprime(2**(bits//2 - 1), 2**(bits//2))
    q = randprime(2**(bits//2 - 1), 2**(bits//2))
    n = p * q
    phi = (p - 1) * (q - 1)
    e = 65537
    d = mod_inverse(e, phi)
    return (e, n), (d, p, q)

def classic_decrypt(c, d, n):
    """
    Классическая расшифровка RSA: m = c^d mod n
    
    Args:
        c: шифротекст (ciphertext)
        d: приватная экспонента (private exponent) 
        n: модуль RSA (modulus)
    
    Returns:
        m: расшифрованное сообщение (plaintext)
    """
    return pow(c, d, n)

# Демонстрация работы
if __name__ == "__main__":
    # Генерируем ключи
    public_key, private_key = generate_rsa_keys()
    e, n = public_key
    d, p, q = private_key
    
    # Тестовое сообщение
    original_message = 42
    print(f"Исходное сообщение: {original_message}")
    
    # Шифрование: c = m^e mod n
    ciphertext = pow(original_message, e, n)
    print(f"Шифротекст: {ciphertext}")
    
    # Расшифровка классическим методом: m = c^d mod n
    decrypted_message = classic_decrypt(ciphertext, d, n)
    print(f"Расшифрованное сообщение: {decrypted_message}")
    
    # Проверка
    print(f"Успех: {original_message == decrypted_message}")

    # Замеряем время выполнения 100 раз
    time_taken = timeit.timeit(
        lambda: classic_decrypt(ciphertext, d, n),
        number=100
    )
    
    print(f"Время для 100 выполнений: {time_taken:.4f} секунд")
    print(f"Среднее время на операцию: {time_taken/100:.6f} секунд")


Исходное сообщение: 42
Шифротекст: 7766394358793188683704848007211542946516103616608340306232824993758916010157352122604010212855113114314062215399605376820281454947048155022871676574203735949690400422938011634608225600286297862760033198224581062744652220908099172322989347992525775944502629574199773249634587332548128184906241188413317256249296439560743508402248618200229622091416585859874771483853230211838089244997132773382701006679906861834163596581609773343526140222800104483408902189631544321813030205787136206211367766194388883881230114656471582738381788529399485298769847604699954260630178660977176692472167345916881591073611007291492838525205
Расшифрованное сообщение: 42
Успех: True
Время для 100 выполнений: 1.5956 секунд
Среднее время на операцию: 0.015956 секунд


In [36]:
def optimized_decrypt(c, d, p, q):
    """
    Оптимизированная расшифровка через КТО
    """
    # mp = c^(d mod (p-1)) mod p
    mp = pow(c, d % (p-1), p)
    
    # mq = c^(d mod (q-1)) mod q
    mq = pow(c, d % (q-1), q)
    
    # m = КТО(mp, mq, p, q)
    n = p * q
    q_inv = pow(q, -1, p)
    m = (mp * q * q_inv + mq * p * pow(p, -1, q)) % n
    
    return m

Формула Китайской теоремы об остатках:

m = (mp × q × (q⁻¹ mod p) + mq × p × (p⁻¹ mod q)) mod n

In [40]:
from sympy import randprime, mod_inverse
import timeit

def generate_rsa_keys(bits=2048):
    # Генерация двух простых чисел p и q
    p = randprime(2**(bits//2 - 1), 2**(bits//2))
    q = randprime(2**(bits//2 - 1), 2**(bits//2))
    
    # Вычисление модуля n и функции Эйлера
    n = p * q
    phi = (p - 1) * (q - 1)
    
    # Выбор публичной экспоненты и вычисление приватной
    e = 65537
    d = mod_inverse(e, phi)
    
    return (e, n), (d, p, q)

def optimized_decrypt(c, d, p, q):
    """
    Оптимизированная расшифровка через КТО
    """
    # mp = c^(d mod (p-1)) mod p
    mp = pow(c, d % (p-1), p)
    
    # mq = c^(d mod (q-1)) mod q
    mq = pow(c, d % (q-1), q)
    
    # m = КТО(mp, mq, p, q)
    n = p * q
    q_inv = pow(q, -1, p)
    m = (mp * q * q_inv + mq * p * pow(p, -1, q)) % n
    
    return m


# Демонстрация работы
if __name__ == "__main__":
    # Генерируем ключи
    public_key, private_key = generate_rsa_keys()
    e, n = public_key
    d, p, q = private_key
    
    # Тестовое сообщение
    original_message = 12345678
    print(f"Исходное сообщение: {original_message}")
    
    # Шифрование: c = m^e mod n
    ciphertext = pow(original_message, e, n)
    print(f"Шифротекст: {ciphertext}")
    
    # Расшифровка оптимизированным методом
    decrypted_message = optimized_decrypt(ciphertext, d, p, q)
    print(f"Расшифрованное сообщение: {decrypted_message}")
    
    # Проверка
    print(f"Успех: {original_message == decrypted_message}")

    # Замеряем время выполнения 100 раз
    time_taken = timeit.timeit(
        lambda: classic_decrypt(ciphertext, d, n),
        number=100
    )
    
    print(f"Время для 100 выполнений: {time_taken:.4f} секунд")
    print(f"Среднее время на операцию: {time_taken/100:.6f} секунд")

Исходное сообщение: 12345678
Шифротекст: 20731510034614290382171355588784925142053584344901970096162173967026351493562141624166026255556656151449497891741316486775389586162281562593287990379766144262154993426715621653193088162556826353494285482199600766982600923497924778663131172331662137042601655489367430715251683624579961128505732280981154831341216048731944539801582630074208171860085014169520090738547254033019614737655861999292336564916497721913081590245511448555664974170666712384735535349803805978654814125378463975122881170310161328782179880741114380925277985480653410014231788932122638605025292721573781577587075711283205797031616907192802402165888
Расшифрованное сообщение: 12345678
Успех: True
Время для 100 выполнений: 1.6105 секунд
Среднее время на операцию: 0.016105 секунд


# Ответ Google AI:

# Использование КТО в OpenSSL для оптимизации RSA

КТО (Китайская теорема об остатках) применяется в OpenSSL для значительного ускорения операций RSA-алгоритма, включая расшифрование и генерацию цифровых подписей, путем разбиения сложных вычислений на более простые компоненты.

## Принцип работы и преимущества

### Большой модуль
- В RSA используется открытый модуль **n**, представляющий собой произведение двух больших секретных простых чисел:  
  `n = p × q`

### Сложность вычислений
- Операции шифрования и расшифрования требуют возведения в степень по модулю **n**
- Работа с огромными числами **n** (тысячи бит) требует значительных вычислительных ресурсов

### Разделение задачи через КТО
КТО позволяет заменить одну сложную операцию по большому модулю **n** на две независимые, параллельно выполняемые операции по меньшим модулям **p** и **q**

### Ускорение вычислений
- Вычисления по модулям **p** и **q** (примерно вдвое меньшим по размеру, чем **n**) выполняются значительно быстрее
- Результаты двух меньших вычислений эффективно объединяются с помощью КТО для получения итогового результата

## Практический результат

Использование КТО в OpenSSL приводит к:
- **Увеличению производительности операций RSA в 3-4 раза**
- Критическому улучшению производительности для высоконагруженных серверов (веб-серверы с HTTPS)
- Обработке миллионов защищенных соединений с оптимальной скоростью
- Прямому влиянию на общую производительность криптографической системы